'gender' : Sexo del estudiante

'parental level of education': Nivel educativo de los padres

'lunch': Si tiene becas comedor

'test preparation course': Asiste a academia

'math score': Puntuación en Mates

'reading score': Comprensión Lectora

'writing score': Puntuación de escritura

En este reto tendrás que predecir si los resultados académicos del alumno están influenciados por el nivel educativo de los padres. Estos tienen un número que representa el siguiente nivel educativo:

high school: 0,

some high school: 1,

some college: 2,

associate's degree: 3,

bachelor's degree: 4,

master's degree: 5


habria que cambiar los numeros de los niveles educativos:
- 0: some high school
- 1: high school
- 2: some college
- 3: associate's degree
- 4: bachelor's degree
- 5: master's degree

clasificacion multiclases

# imports

Librerías

In [1]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 6.0 MB/s eta 0:00:00


In [2]:
# cálculos
import numpy as np
import pandas as pd

# fechas
from datetime import datetime

# gráficos
import seaborn as sns
import matplotlib.pyplot as plt

# preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures

# algoritmos de clasificación
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# algoritmos de regresión
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

# métricas
import sklearn.metrics as metrics

# hiperparametrización
from sklearn.model_selection import GridSearchCV

# TensorFlow y tf.keras
import tensorflow as tf
from tensorflow import keras

import xgboost as xgb

from numpy import random
random.seed(42)

datos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df_entrenar = pd.read_csv("/content/drive/MyDrive/Machine Learning/multiclases entrenar.csv")
df_predecir = pd.read_csv("/content/drive/MyDrive/Machine Learning/multiclases predecir.csv")

# comprender datos

In [ ]:
df_entrenar

In [ ]:
df_predecir

In [ ]:
df_entrenar.info()

In [ ]:
#El select_dtypes sirve para: Seleccionamos un subdataframe con las categorias que elijamos. Si queremos que nos muestre todas las columnas type object, nos las muestra.
df_entrenar.select_dtypes("object").columns.to_list()

In [ ]:
# eliminamos la columna Unnamed: 0
df_entrenar.drop('Unnamed: 0', axis=1, inplace=True)
df_predecir.drop('Unnamed: 0', axis=1, inplace=True)

#Borramos esa columna porque no es necesaria y porque si luego eliminamos filas porque hay nulos o por lo que sea el orden o los numeros de esa columna Unnamed se pierde o se cambia.

In [ ]:
# cambiamos el nombre de las columanas para que sea más fácil usarlas
columnas = df_entrenar.columns.str.replace(" ","_")
df_entrenar.columns = columnas
columnas = df_predecir.columns.str.replace(" ","_")
df_predecir.columns = columnas

In [ ]:
df_entrenar.info()

In [ ]:
df_entrenar.nunique()

In [ ]:
df_entrenar['gender'].unique()

In [ ]:
df_entrenar['lunch'].unique()

In [ ]:
df_entrenar['test_preparation_course'].unique()

In [ ]:
df_entrenar['parental_level_of_education'].unique()

In [ ]:
df_entrenar['math score'].unique()

In [ ]:
df_entrenar.isnull().sum()

In [ ]:
df_predecir.isnull().sum()

# análisis exploratorio

In [ ]:
df_entrenar.head()

In [ ]:
# cambio de las etiquetas 0 y 1 para tener un orden en el nivel de education
# "some high school" 1 es más bajo que "high school" 0. Lo intercamiamos
#Basicamente esto lo hacemos porque tiene mas sentido que la gemte con some hight school tenga el nº 0 y la gente con el hight school tenga el nº1 y no al revés, porque no tiene snetido.
dict = {
    0: 1,
    1: 0,
    2: 2,
    3: 3,
    4: 4,
    5: 5
}

df_entrenar['parental_level_of_education'] = df_entrenar['parental_level_of_education'].map(dict)

# otra forma
# df_entrenar['parental_level_of_education'] = df_entrenar['parental_level_of_education'].apply(lambda x: 0 if x == 1
#                                                                                                         else (1 if x == 0 else x))

In [ ]:
df_entrenar.info()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 3))
sns.countplot(data=df_entrenar, x='parental_level_of_education')
plt.show()

In [ ]:
sns.scatterplot(data=df_entrenar, x='parental_level_of_education', y='math_score')
plt.show()

In [ ]:
# jittering: agregar un pequeño valor aleatorio a los datos categóricos para dispersarlos, transformandolos en una variable continúa.
# usar solo para los gráficos!
df_entrenar['target_jittered'] = np.random.normal(df_entrenar['parental_level_of_education'], scale=0.3)
#Lo del scale=0.3 es para separar más o menos los puntos, si pones 0.5 separa más los puntos que si pones 0.1

In [ ]:
sns.scatterplot(data=df_entrenar, x='target_jittered', y='math_score')
plt.show()

In [ ]:
sns.scatterplot(data=df_entrenar, x='target_jittered', y='reading_score')
plt.show()

In [ ]:
sns.scatterplot(data=df_entrenar, x='target_jittered', y='writing_score')
plt.show()

In [ ]:
df_entrenar.drop('target_jittered', axis=1, inplace=True)

# preprocessing

In [ ]:
# nulos
df_entrenar.isnull().sum()

In [ ]:
# one hot encoding
df_entrenar = pd.get_dummies(df_entrenar)
df_predecir = pd.get_dummies(df_predecir)

In [ ]:
df_entrenar

In [ ]:
# eliminamos las variables sobrantes: una de cada 2 que se han creado de las variables con 2 categorías
df_entrenar.drop(['gender_male', 'lunch_standard', 'test_preparation_course_none'], axis=1, inplace=True)
df_predecir.drop(['gender_male', 'lunch_standard', 'test_preparation_course_none'], axis=1, inplace=True)
#Da igual que te quedes con una columna o con la otra porque por ejemplo, si una persona es Male es evidente que no será Female. Basicamente te sobra una de los dos columnas

In [ ]:
plt.rcParams['figure.figsize'] = 8, 6
sns.heatmap(df_entrenar.corr(), annot=True, cmap="YlGnBu")
plt.show()

In [ ]:
#Después de hacer las correlaciones vemos que hay dos columnas que tienen una correlacion de 0.95, nos podemos cargar una de las dos columnas. Nos quedamos con la columna que está mas correlacionada con la Target, en este caso te quedas con writing_score.

In [ ]:
# eliminamos reading_score por tener una correlación muy alta con writing_score y porque aporta menos información que writing_score (menos correlación con la target)
df_entrenar.drop(['reading_score'], axis=1, inplace=True)
df_predecir.drop(['reading_score'], axis=1, inplace=True)

In [ ]:
#feature importance
params = {'random_state': 42, 'n_jobs': 4, 'n_estimators': 100, 'max_depth': 4}
# se entrena un RandomForest y se plotean las variables que tuvieron más importancia en el modelo
y = df_entrenar['parental_level_of_education']
x = df_entrenar.drop('parental_level_of_education' ,axis=1)
# entrena un RandomForest Classifier
clf = RandomForestClassifier(**params)
clf = clf.fit(x, y)
# Plotea las 10 mejores Features importances
features = clf.feature_importances_[:10]
columnas = x.columns[:10]
imp = pd.Series(data=features, index=columnas).sort_values(ascending=False)
plt.figure(figsize=(10,8))
plt.title("Feature importance")
ax = sns.barplot(y=imp.index, x=imp.values, palette="Blues_d", orient='h')
plt.show()

# modelo

## split train/test

In [ ]:
# se guarda un 30% de datos para el test y un 70% para el train
y = df_entrenar['parental_level_of_education']
X = df_entrenar.drop('parental_level_of_education', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#Basicamente el df_entrenar lo tenemos que dividir en 2, una parte que se ael 70% para entrenar y otro 30% para evaluar si el entrenamiento del 70% ha sido bueno.
#OJO!! No confundir con df_predecir

## estandarización

In [ ]:
# se estandarizan las variables para evitar sesgo
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)   # estandariza y define la regla de estandarización
X_test = scaler.transform(X_test)         # estandariza según la regla definida con el X_train

# se estandarizan las variables del dataset a predecir según la regla definida con el X_train
X_predecir = scaler.transform(df_predecir)

In [ ]:
X_train.shape

In [ ]:
f1s = pd.DataFrame(columns=['modelo', 'f1_score'])
f1s
#f1_score tiene en cuenta los falsos positivos y los falsos negativos. Es cuanto has acertado teniendo en cuenta los falsos positivos y negativos

## clasificación multiclase

In [ ]:
# añadimos en una lista los modelos que queremos poner a competir

modelos = []
modelos.append(('SVC', SVC()))
modelos.append(('LogisticRegression', LogisticRegression()))
modelos.append(('RidgeClassifier', RidgeClassifier()))
modelos.append(('DecisionTreeClassifier', DecisionTreeClassifier()))
modelos.append(('RandomForestClassifier', RandomForestClassifier()))
modelos.append(('GradientBoostingClassifier', GradientBoostingClassifier()))
modelos.append(('XGBClassifier', XGBClassifier()))
modelos.append(('CatBoostClassifier', CatBoostClassifier(verbose=False)))

In [ ]:
for nombre, modelo in modelos:
  # entrenamiento
  modelo.fit(X_train, y_train)

  # predicción del test
  y_pred = modelo.predict(X_test)

  # evalua la metrica
  f1_score = metrics.f1_score(y_test, y_pred, average='macro')
  #el average='macro' se usa cuando estamos haciendo modelos multiclases. Sino lo pones el modelo peta.

  # añade la métrica al dataframe de métricas
  metrica = pd.DataFrame({'modelo':[nombre], 'f1_score':[f1_score]})

  print(nombre, np.round(f1_score,5))

  f1s = pd.concat([f1s,metrica], ignore_index=True)

In [ ]:
f1s.sort_values('f1_score', ascending= False)

In [ ]:
  #Escogemos el modelo con el nº más alto

## regresión

In [ ]:
# añadimos en una lista los modelos que queremos poner a competir

modelos = []
modelos.append(('LinearRegression', LinearRegression()))
modelos.append(('Lasso', Lasso()))
modelos.append(('Ridge', Ridge()))
modelos.append(('ElasticNet', ElasticNet()))
modelos.append(('SGDRegressor', SGDRegressor()))
modelos.append(('SVC', SVC()))
modelos.append(('RandomForestRegressor', RandomForestRegressor()))
modelos.append(('GradientBoostingRegressor', GradientBoostingRegressor()))
modelos.append(('XGBRegressor', XGBRegressor(objective ='reg:squarederror')))
modelos.append(('CatBoostRegressor', CatBoostRegressor(verbose=False)))

In [ ]:
for nombre, modelo in modelos:
  # entrenamiento
  modelo.fit(X_train, y_train)

  # predicción del test
  y_pred = modelo.predict(X_test)

  # pasamos las predicciones dataframe
  df_pred = pd.DataFrame(y_pred, columns=['prediccion'])

  # redeondeamos las predicciones
  df_pred['prediccion'] = round(df_pred['prediccion'], 0)

  # cambiamos los valores extremos
  df_pred['prediccion'] = df_pred['prediccion'].apply(lambda x: 0 if x < 0
                                                                else (5 if x > 5
                                                                      else int(x)))

  #lambda aplica una columna a cada fila del dataframe y le aplica una función.

  # evalua la metrica
  f1_score = metrics.f1_score(y_test, df_pred['prediccion'], average='macro')

  # añade la métrica al dataframe de métricas
  metrica = pd.DataFrame({'modelo':[nombre], 'f1_score':[f1_score]})

  print(nombre, np.round(f1_score,5))

  f1s = pd.concat([f1s,metrica], ignore_index=True)

In [ ]:
f1s.sort_values('f1_score', ascending= False)

XGBClassifier	0.227501  
LogisticRegression	0.206864  
DecisionTreeClassifier	0.205452  
CatBoostRegressor	0.201459  
RidgeClassifier	0.196502  
RandomForestClassifier	0.192750  
CatBoostClassifier	0.190005  
XGBRegressor	0.177721  
RandomForestRegressor	0.168087  
SVC	0.163345  
SVC	0.163345  
GradientBoostingRegressor	0.162172  
GradientBoostingClassifier	0.160278  
Ridge	0.154498  
LinearRegression	0.154351  
SGDRegressor	0.139899  
ElasticNet	0.076923  
Lasso	0.076923  

# predicción

Usaremos el mejor modelo XGBClassifier

In [ ]:
# definimos el modelo
modelo = XGBClassifier()

# entrenamiento
modelo.fit(X_train, y_train)

# predicción de las observaciones a predecir
y_pred = modelo.predict(X_predecir)

# añadimos la predicción al dataframe a predecir
df_predecir['prediccion'] = y_pred

# redeondeamos las predicciones
df_predecir['prediccion'] = round(df_predecir['prediccion'], 0)

# cambiamos los valores extremos
df_predecir['prediccion'] = df_predecir['prediccion'].apply(lambda x: 0 if x < 0
                                                                      else (5 if x > 5
                                                                            else int(x)))

In [ ]:
len (df_predecir)

In [ ]:
df_predecir